In [ ]:
import importlib.metadata
importlib.metadata.version("snowflake-ml-python")

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# Suppose your table is IRIS_DATA
df = session.table("IRIS_DB.PUBLIC.IRIS")

# Rename columns to simple lowercase names
df_fixed = df.select(
    df["SEPAL_LENGTH"].cast("DOUBLE").alias("sepal_length"),
    df["SEPAL_WIDTH"].cast("DOUBLE").alias("sepal_width"),
    df["PETAL_LENGTH"].cast("DOUBLE").alias("petal_length"),
    df["PETAL_WIDTH"].cast("DOUBLE").alias("petal_width"),
    df["VARIETY"].alias("variety")
)

df_fixed.show()

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()

df = session.table("IRIS_DB.PUBLIC.IRIS")

df_fixed = df.select(
    df["SEPAL_LENGTH"].cast("DOUBLE").alias("sepal_length"),
    df["SEPAL_WIDTH"].cast("DOUBLE").alias("sepal_width"),
    df["PETAL_LENGTH"].cast("DOUBLE").alias("petal_length"),
    df["PETAL_WIDTH"].cast("DOUBLE").alias("petal_width"),
    df["VARIETY"].alias("label")
)

df_fixed.show()

In [ ]:
train_df, test_df = df_fixed.random_split([0.8, 0.2], seed=42)

print("Train count:", train_df.count())
print("Test count:", test_df.count())

In [ ]:
FEATURE_COLS = [
    "sepal_length",
    "sepal_width",
    "petal_length",
    "petal_width"
]

LABEL_COL = "label"

In [ ]:
df_fixed.write.mode("overwrite").save_as_table(
    "IRIS_DB.PUBLIC.IRIS_FEATURES_TEMP",
    table_type="temporary"
)

In [ ]:
features_df = session.table("IRIS_DB.PUBLIC.IRIS_FEATURES_TEMP")
features_df.show()

In [ ]:
train_df, test_df = features_df.random_split([0.8, 0.2], seed=42)

In [ ]:
features_df.schema

In [ ]:
from snowflake.ml.modeling.linear_model import LogisticRegression

model = LogisticRegression(
    input_cols=[
        "SEPAL_LENGTH",
        "SEPAL_WIDTH",
        "PETAL_LENGTH",
        "PETAL_WIDTH"
    ],
    label_cols="LABEL",
    output_cols="PREDICTION"
)

model.fit(train_df)


In [ ]:
predictions = model.predict(test_df)

predictions.select(
    "SEPAL_LENGTH",
    "SEPAL_WIDTH",
    "PETAL_LENGTH",
    "PETAL_WIDTH",
    "LABEL",
    "PREDICTION"
).show()


In [ ]:
from snowflake.ml.registry import Registry

registry = Registry(
    session=session,
    database_name="IRIS_DB",
    schema_name="PUBLIC"
)


In [ ]:
model_version = registry.log_model(
    model=model,
    model_name="IRIS_CLASSIFIER",
    version_name="V1",
    sample_input_data=train_df.limit(5)
)

In [ ]:
registry.get_model("IRIS_CLASSIFIER")

In [ ]:
registry.get_model("IRIS_CLASSIFIER").versions()

In [ ]:
loaded_model = registry.get_model("IRIS_CLASSIFIER").version("V1")

In [ ]:
inference_df = loaded_model.run(
    test_df,
    function_name="predict"
)

In [ ]:
inference_df.select(
    "SEPAL_LENGTH",
    "SEPAL_WIDTH",
    "PETAL_LENGTH",
    "PETAL_WIDTH",
    "LABEL",
    "PREDICTION"
).show()
